In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, ConstantKernel

## Read and Reformat the data

In [2]:
data_file = "../data/220720PM25diffsite.csv"
output_file = "../result/gpr_all_features.csv"

df = pd.read_csv(data_file)
selected_features = ["pm25", "ENSOmonthly"
           ,"eNOx","SO2emis","PM25emis","eVOC","NH3emis"
           ,"TMAXbarstow","AWNDLAX","Mir850RH","Rhontario"
           ,"dayofweekf","dayofyear"]
all_features = ["pm25", "ENSOmonthly"
              ,"eNOx","SO2emis","PM25emis","eVOC","NH3emis"
              ,"TMAXbarstow","AWNDLAX","Mir850RH","Rhontario"
              ,"dayofweekf","dayofyear"
              ,"MirTemp500C","MirWS850ms","MirWD850","MirHeight850","MirWS500ms","MirWD500","Mir500RH"
              ,"SRmeanC","AWNDbarstow","TMAXLAX","TMAXontario","AWNDontario"]
df_selected = df[selected_features]
df_all_features = df[all_features]

In [3]:
dataset = df_all_features.dropna()
label_name = "pm25"
y_vector = dataset[[label_name]]
# change it for all features or selected features
features_names = all_features.copy()
# features_names = selected_features.copy()
features_names.remove(label_name)
X_matrix = dataset[features_names]

In [4]:
def dayofweekToNum(data_frame):
    day_mapping = {"Mon": 1, "Tue": 2, "Wed": 3, "Thu": 4, "Fri": 5, "Sat": 6, "Sun": 7}
    dayofweekf = data_frame["dayofweekf"].to_numpy()
    res = []
    for i in range(0, len(dayofweekf)):
        res.append(day_mapping[dayofweekf[i]])
    data_frame.loc[:, ("dayofweekf")] = res
    return data_frame
X_matrix = dayofweekToNum(X_matrix)
print(X_matrix)

      ENSOmonthly      eNOx  SO2emis  PM25emis     eVOC  NH3emis  TMAXbarstow  \
3           24.78  1007.938   62.837    78.766  999.205   94.759         16.7   
4           24.78  1007.938   62.837    78.766  999.205   94.759         16.7   
6           24.78  1007.938   62.837    78.766  999.205   94.759         13.3   
8           24.78  1007.938   62.837    78.766  999.205   94.759         18.3   
9           24.78  1007.938   62.837    78.766  999.205   94.759         23.9   
...           ...       ...      ...       ...      ...      ...          ...   
7299        27.07   337.141   16.174    81.619  526.083   79.151          7.0   
7300        27.07   337.141   16.174    81.619  526.083   79.151          9.0   
7301        27.07   337.141   16.174    81.619  526.083   79.151         11.0   
7302        27.07   337.141   16.174    81.619  526.083   79.151          9.0   
7303        27.07   337.141   16.174    81.619  526.083   79.151          9.0   

      AWNDLAX   Mir850RH  R

/Users/zongrunli/opt/anaconda3/envs/Data_Analysis/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [5]:
features_data = X_matrix.to_numpy()
_, num_features = features_data.shape
label_data = y_vector.to_numpy()
# split the data for 10-fold cross validation
kf = KFold(n_splits=10, shuffle=True, random_state=100)

## Gaussian Process Regression

In [6]:
# for selected feature 19, 105, for all feature we still use 19, 105
hyper_parameters = [(19, 105)]

In [7]:
testing_data_rows = []
training_data_rows = []
final_r2 = 0
final_rmse = 0
final_mbe = 0
final_prediction = None
for hyper_parameter in hyper_parameters:
    r2_res = []
    rmse_res = []
    r2_res_training = []
    rmse_res_training = []
    for train_index, test_index in kf.split(features_data):
        X_train, X_test = features_data[train_index], features_data[test_index]
        y_train, y_test = label_data[train_index], label_data[test_index]
        kernel = ConstantKernel(hyper_parameter[0], 'fixed') * RBF(hyper_parameter[1], 'fixed')
        model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1, alpha=0.1)
        model.fit(X_train, y_train)
        # evaluate the model performance
        # test data
        predict_res = model.predict(X_test)
        r2_res.append(r2_score(y_test, predict_res))
        rmse_res.append(mean_squared_error(predict_res, y_test, squared=False))
        # training data
        predict_res = model.predict(X_train)
        r2_res_training.append(r2_score(y_train, predict_res))
        rmse_res_training.append(mean_squared_error(predict_res, y_train, squared=False))
    print(r2_res)
    # write down the performance for current hyperparameters
    r2_mean = np.mean(r2_res)
    rmse_mean = np.mean(rmse_res)
    data_row = [hyper_parameter[0], hyper_parameter[1], r2_mean, rmse_mean]
    testing_data_rows.append(data_row)
    
    r2_mean = np.mean(r2_res_training)
    rmse_mean = np.mean(rmse_res_training)
    data_row = [hyper_parameter[0], hyper_parameter[1], r2_mean, rmse_mean]
    training_data_rows.append(data_row)
    # train by all data
    kernel = ConstantKernel(hyper_parameter[0], 'fixed') * RBF(hyper_parameter[1], 'fixed')
    model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1, alpha=0.1)
    model.fit(features_data, label_data)
    # evaluate the model performance
    predict_res = model.predict(features_data)
    final_r2 = r2_score(label_data, predict_res)
    final_rmse = mean_squared_error(predict_res, label_data, squared=False)
    final_mbe = np.mean(predict_res - label_data)
    final_prediction = predict_res

[0.6968301063049521, 0.6549642913868534, 0.6361694115602352, 0.6468672680288066, 0.6182955664003815, 0.6801774519936346, 0.5329498257309033, 0.6171227602829057, 0.6766846684128364, 0.6264682920951252]


## Cross Validation Results

In [8]:
# for Table S4
print("Training Data")
print("R2 = %f   RMSE = %f" %(training_data_rows[0][2], training_data_rows[0][3]))
print("Testing Data")
print("R2 = %f   RMSE = %f" %(testing_data_rows[0][2], testing_data_rows[0][3]))

Training Data
R2 = 0.897920   RMSE = 4.000951
Testing Data
R2 = 0.638653   RMSE = 7.511375


## Final Model Performance

In [9]:
# for Table 1
print("R2 = %f   RMSE = %f    MBE = %f" %(final_r2, final_rmse, final_mbe))

R2 = 0.893463   RMSE = 4.087599    MBE = -0.001066


## Generate Data for Annual Evaluation

In [10]:
X_matrix['predict_values'] = final_prediction
# save to csv
X_matrix.to_csv(output_file, index=False)